<a href="https://colab.research.google.com/github/wdoppenberg/mp-operational-envelope/blob/master/OperationalEnvelope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/wdoppenberg/mp-operational-envelope.git

Cloning into 'mp-operational-envelope'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 52 (delta 15), reused 44 (delta 11), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [6]:
cd mp-operational-envelope

/content/mp-operational-envelope


In [0]:
import numpy as np
from analysis import constants as c, functions as f, components as cmp

import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [0]:
def mdot(p, A_t, Gamma, R, T_c):
    """
    Mass flow function.

    Accepts type `np.array` for `p` and `T_c`
    """
    return (p*A_t*Gamma) / np.sqrt(R*T_c)

def p_t(V_0, p_0, m_exit_t, rho):
    """
    Pressure vs time function.
    """
    return (V_0 * p_0) / (V_0 + (m_exit_t/rho))

In [0]:
p_0 = np.arange(1., 2.2, 0.2)*1e5
T_c0 = c.h_vap*c.T_vap0/(c.T_vap0*c.R_vap*np.log(c.p_vap0/p_0)+c.h_vap)
m_0 = p_0*c.A_t*c.Gamma/(np.sqrt(c.R*T_c0))
V_0 = np.arange(0.05, 0.2, 0.05)*c.V_tube

sim_dim = (len(p_0), len(V_0), len(c.t))

m = np.zeros(sim_dim)
p = np.zeros(sim_dim)
temp = np.zeros(sim_dim)

m[:, :, 0], _ = np.meshgrid(m_0, V_0, indexing='ij')
p[:, :, 0], V_0 = np.meshgrid(p_0, V_0, indexing='ij')
p_0 = p[:, :, 0]

T_c = 600

In [10]:
p_0

array([[100000., 100000., 100000., 100000.],
       [120000., 120000., 120000., 120000.],
       [140000., 140000., 140000., 140000.],
       [160000., 160000., 160000., 160000.],
       [180000., 180000., 180000., 180000.],
       [200000., 200000., 200000., 200000.],
       [220000., 220000., 220000., 220000.]])

In [0]:
%%time
for ii in range(1,len(c.t)):
    p[:,:,ii] = V_0*p_0/(V_0+temp[:,:,ii-1])
    m[:,:,ii] = mdot(p[:,:,ii-1], c.A_t, c.Gamma, c.R, T_c)
    temp[:,:,ii] = temp[:,:,ii-1]+m[:,:,ii]*c.dt/c.rho

T_vap = c.h_vap*c.T_vap0/(c.T_vap0*c.R_vap*np.log(c.p_vap0/p)+c.h_vap)

CPU times: user 804 ms, sys: 21.9 ms, total: 826 ms
Wall time: 838 ms


In [0]:
p

array([[[100000.        , 100000.        ,  99802.17216446, ...,
           9994.61296341,   9994.41506361,   9994.21717557],
        [100000.        , 100000.        ,  99900.98814573, ...,
          14064.58674202,  14064.31100293,  14064.03528006],
        [100000.        , 100000.        ,  99933.97030474, ...,
          17141.07035591,  17140.73758913,  17140.40484174],
        [100000.        , 100000.        ,  99950.46955236, ...,
          19696.66419625,  19696.28552228,  19695.90687015]],

       [[120000.        , 120000.        , 119715.2405835 , ...,
          10957.64580618,  10957.42847705,  10957.21116086],
        [120000.        , 120000.        , 119857.4511579 , ...,
          15432.40247901,  15432.09892443,  15431.79538777],
        [120000.        , 120000.        , 119904.92979374, ...,
          18823.19759236,  18822.83037339,  18822.46317591],
        [120000.        , 120000.        , 119928.6832201 , ...,
          21646.65636718,  21646.23749777,  21645.8

In [0]:
fig, ax = plt.subplots(1,1)

for i in p:
    ax.plot(c.t, i[:])

ax.set_xlabel('Time [s]')
ax.set_ylabel('Pressure [Pa]')
ax.text(1800, 1.6e5, '$V_0 = 0.1*V_{tube}$')
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.legend()
ax.grid()

ValueError: x and y must have same first dimension, but have shapes (25000,) and (4,)

In [0]:
np.allclose(p, p2)

True